# Z-Score Normalization

[]

In [11]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel # 필요한 라이브러리 임포트
from sklearn.feature_extraction.text import TfidfVectorizer # 태그 벡터화 라이브러리 임포트

DATA_PATH = './data'
ratings = pd.read_csv(f'{DATA_PATH}/ratings.csv')
movies  = pd.read_csv(f'{DATA_PATH}/movies.csv')

SEED_LIST = [10, 21, 35, 42, 57]
PERSONA_HEAVY_USER = 414
PERSONA_GENRE_SPECIALIST = 85

# [Tag Preprocessing]
tags_df = pd.read_csv(f'{DATA_PATH}/tags.csv')
tags_df['tag'] = tags_df['tag'].astype(str).str.lower()
movie_tags_series = tags_df.groupby('movieId')['tag'].apply(lambda x: ' '.join(x))

print("태그 데이터 로드 및 전처리 완료")

태그 데이터 로드 및 전처리 완료


In [12]:
def calculate_genre_similarity(movies_df):
    """ movies DataFrame을 사용하여 영화 간 장르 코사인 유사도 행렬을 계산합니다. """
    movies_df = movies_df.copy()
    movies_df['genres_list'] = movies_df['genres'].apply(lambda x: x.split('|'))
    
    mlb = MultiLabelBinarizer()
    genre_matrix = pd.DataFrame(mlb.fit_transform(movies_df['genres_list']),
                                columns=mlb.classes_,
                                index=movies_df['movieId'])
    
    if '(no genres listed)' in genre_matrix.columns:
        genre_matrix = genre_matrix.drop(columns=['(no genres listed)'])

    # 코사인 유사도 = 정규화된 Dot Product
    S_genre_array = cosine_similarity(genre_matrix.values)
    
    S_genre_df = pd.DataFrame(S_genre_array, 
                              index=genre_matrix.index, 
                              columns=genre_matrix.index)
    
    np.fill_diagonal(S_genre_df.values, 0.0)
    
    return S_genre_df

## 1. 전처리 함수 (Z-Score + 최소 평점 필터링)

In [13]:
def run_preprocessing(train_df, test_df):
    train = train_df.copy()
    test  = test_df.copy()

    # 1. 최소 5회 이상 평점 준 사용자만 사용 (페르소나는 강제 포함)
    user_counts = train['userId'].value_counts()
    valid_users = set(user_counts[user_counts >= 5].index)
    valid_users = valid_users.union({PERSONA_HEAVY_USER, PERSONA_GENRE_SPECIALIST})

    train = train[train['userId'].isin(valid_users)]
    test  = test[test['userId'].isin(valid_users)]

    # 2. Train에서만 사용자별 mean / std 계산
    user_stats = train.groupby('userId')['rating'].agg(['mean', 'std']).fillna(1.0)
    user_stats['std'] = user_stats['std'].replace(0, 1.0)   # std=0 방지

    # 3. Z-Score 변환 (Train)
    train = train.merge(user_stats, on='userId', suffixes=('', '_user'))
    train['z_rating'] = (train['rating'] - train['mean']) / train['std']

    # 4. Test도 동일한 통계량으로 변환 (Train에 없는 사용자는 제거 → cold-start 방지)
    test = test.merge(user_stats, on='userId', how='left')
    test = test.dropna(subset=['mean', 'std'])   # Train에 없는 사용자 제거
    test['z_rating'] = (test['rating'] - test['mean']) / test['std']

    return train, test, user_stats   # user_stats도 반환 (역변환에 필요)

## 2. 평가 함수

In [14]:
import numpy as np
import time

def fast_predict_matrix_no_k(train_df):
    """
    K 제한 없이, 유사도가 양수인 모든 아이템을 사용하여 예측
    """
    # 1. Pivot Table 생성 및 R 행렬 정의
    pivot_df = train_df.pivot(index='userId', columns='movieId', values='z_rating').fillna(0)
    R = pivot_df.values  # (Users, Items)
    user_ids = pivot_df.index
    movie_ids = pivot_df.columns
    
    # 2. 아이템 간 유사도 행렬 계산
    sim_matrix = np.dot(R.T, R)
    np.fill_diagonal(sim_matrix, -np.inf)
    
    # 3. [핵심 변경] Top-K 필터링 로직 삭제! 양수 유사도만 남김
    S = np.where(sim_matrix > 0, sim_matrix, 0)

    # 4. 예측 평점 계산 (Weighted Sum) -> 분자
    numerator = np.dot(R, S)
    
    # 5. 가중치 합 계산 (Sum of Weights) -> 분모
    R_binary = (R != 0).astype(float)
    denominator = np.dot(R_binary, np.abs(S))
    
    # 6. 나눗셈 (0으로 나누기 방지)
    with np.errstate(divide='ignore', invalid='ignore'):
        pred_z = numerator / denominator
        pred_z = np.nan_to_num(pred_z)
        
    return pred_z, user_ids, movie_ids

In [15]:
def fast_predict_matrix_triple_hybrid(train_df, S_genre_df, movie_tags_series, alpha=0.8, beta=0.1):
    """
    평점(Rating), 장르(Genre), 태그(Tag) 3가지 유사도를 결합한 하이브리드 예측.
    S_final = alpha * S_rating_norm + beta * S_genre + (1 - alpha - beta) * S_tag
    """
    # 1. Pivot Table 및 R 행렬 정의
    pivot_df = train_df.pivot(index='userId', columns='movieId', values='z_rating').fillna(0)
    R = pivot_df.values
    user_ids = pivot_df.index
    movie_ids = pivot_df.columns
    gamma = 1 - alpha - beta # 태그 가중치 (나머지)
    
    # -------------------------------------------------------
    # [A] 평점 기반 유사도 (S_rating)
    # -------------------------------------------------------
    sim_rating = np.dot(R.T, R)
    max_rating = np.max(np.abs(sim_rating)) if np.max(np.abs(sim_rating)) > 0 else 1.0
    S_rating_norm = sim_rating / max_rating # -1 ~ 1 사이 정규화
    
    # -------------------------------------------------------
    # [B] 장르 유사도 (S_genre) - Alignment
    # -------------------------------------------------------
    S_genre_aligned = S_genre_df.loc[movie_ids, movie_ids].values

    # -------------------------------------------------------
    # [C] 태그 유사도 (S_tag) - Recalculation & Alignment (최적 max_features 500 사용)
    # -------------------------------------------------------
    current_movie_tags = [movie_tags_series.get(mid, "") for mid in movie_ids]
    tfidf = TfidfVectorizer(stop_words='english', max_features=500) 
    tfidf_matrix = tfidf.fit_transform(current_movie_tags)
    S_tag = linear_kernel(tfidf_matrix, tfidf_matrix)

    # -------------------------------------------------------
    # [D] 3중 유사도 결합 (Triple Hybrid Fusion)
    # -------------------------------------------------------
    sim_final = alpha * S_rating_norm + beta * S_genre_aligned + gamma * S_tag
    
    # 자기 자신과의 유사도 제거 및 양수 필터링
    np.fill_diagonal(sim_final, -np.inf)
    S = np.where(sim_final > 0, sim_final, 0)

    # 5. 예측 계산 (Weighted Sum)
    numerator = np.dot(R, S)
    R_binary = (R != 0).astype(float)
    denominator = np.dot(R_binary, np.abs(S))
    
    with np.errstate(divide='ignore', invalid='ignore'):
        pred_z = numerator / denominator
        pred_z = np.nan_to_num(pred_z)
        
    return pred_z, user_ids, movie_ids

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel # Dot Product와 동일

def fast_predict_matrix_with_tags(train_df, movie_tags_series, tag_weight=0.15):
    """
    평점 기반 CF와 태그 기반 Content-based Filtering을 결합한 하이브리드 예측
    
    Args:
        tag_weight (float): 태그 유사도의 반영 비율 (0.0 ~ 1.0)
                            0.0이면 순수 CF, 1.0이면 순수 태그 추천
    """
    # 1. Pivot Table 생성 및 R 행렬 정의 (Z-Score)
    pivot_df = train_df.pivot(index='userId', columns='movieId', values='z_rating').fillna(0)
    R = pivot_df.values
    user_ids = pivot_df.index
    movie_ids = pivot_df.columns # 현재 Train 셋에 존재하는 영화 ID 순서
    
    # -------------------------------------------------------
    # [A] 평점 기반 유사도 (Collaborative Filtering) - Dot Product
    # -------------------------------------------------------
    sim_cf = np.dot(R.T, R)
    
    # -------------------------------------------------------
    # [B] 태그 기반 유사도 (Content-Based) - Dot Product (Linear Kernel)
    # -------------------------------------------------------
    # 1. 현재 Train에 있는 영화 순서대로 태그 데이터 정렬 (없으면 빈 문자열)
    current_movie_tags = [movie_tags_series.get(mid, "") for mid in movie_ids]
    
    # 2. TF-IDF 벡터화
    tfidf = TfidfVectorizer(stop_words='english', max_features=500)
    tfidf_matrix = tfidf.fit_transform(current_movie_tags)
    
    # 3. 태그 유사도 계산 (Dot Product)
    # TF-IDF 벡터는 정규화되어 있으므로 Dot Product는 코사인 유사도와 같음
    sim_tags = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    # -------------------------------------------------------
    # [C] 유사도 결합 (Hybrid) & 스케일 맞추기
    # -------------------------------------------------------
    # 평점 유사도(sim_cf)는 값이 크고(예: 수백), 태그 유사도(sim_tags)는 작음(0~1)
    # 따라서 sim_cf를 최대값으로 나누어 -1 ~ 1 사이로 정규화한 뒤 결합
    
    max_cf = np.max(np.abs(sim_cf)) if np.max(np.abs(sim_cf)) > 0 else 1.0
    sim_cf_norm = sim_cf / max_cf 
    
    # 하이브리드 유사도 계산
    # (1 - tag_weight) * 평점유사도 + (tag_weight) * 태그유사도
    sim_hybrid = (1 - tag_weight) * sim_cf_norm + (tag_weight) * sim_tags
    
    # 자기 자신과의 유사도 제거
    np.fill_diagonal(sim_hybrid, -np.inf)
    
    # 4. 양수 유사도만 남김 (K 제한 없음)
    S = np.where(sim_hybrid > 0, sim_hybrid, 0)

    # 5. 예측 평점 계산 (Weighted Sum)
    numerator = np.dot(R, S)
    
    R_binary = (R != 0).astype(float)
    denominator = np.dot(R_binary, np.abs(S))
    
    with np.errstate(divide='ignore', invalid='ignore'):
        pred_z = numerator / denominator
        pred_z = np.nan_to_num(pred_z)
        
    return pred_z, user_ids, movie_ids

In [17]:
def rmse(y_true, y_pred): return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))
def mae(y_true, y_pred):  return np.mean(np.abs(np.array(y_true)-np.array(y_pred)))

def time_predictions(fn, pairs):
    s = time.perf_counter()
    preds = [fn(u,i) for u,i in pairs]
    e = time.perf_counter() - s
    return preds, e, (e/len(pairs))*1000

# 최적 파라미터 찾기

In [18]:
# ==========================================
# [Grid Search] 최적의 α (평점), β (장르), γ (태그) 가중치 찾기
# ==========================================
from sklearn.metrics.pairwise import linear_kernel
import time
import pandas as pd

# 1. 테스트할 파라미터 후보군 정의
# α (평점 가중치): CF가 중요하므로 높게 시작
ALPHA_LIST = [0.7, 0.8, 0.9]
# β (장르 가중치): 0.05 ~ 0.15 사이를 탐색
BETA_LIST = [0.05, 0.1, 0.15] 

# 결과 저장용
grid_results = []

# 2. 필수 데이터/함수 초기화 (Grid Search 루프 밖에서 1회 실행)
# 주의: calculate_genre_similarity 함수와 movie_tags_series는 이전에 정의/로드되어 있어야 합니다.
FIXED_SEED = 42 # 일관된 비교를 위해 고정 시드 사용
S_genre_df = calculate_genre_similarity(movies.copy()) 

train_raw, test_raw = train_test_split(ratings, test_size=0.2, random_state=FIXED_SEED)
train, test, user_stats = run_preprocessing(train_raw, test_raw)

print("=== Grid Search 시작 (고정 시드 42 사용) ===")

# 예측 및 평가를 위한 기본 설정
pivot_temp = train.pivot(index='userId', columns='movieId', values='z_rating').fillna(0)
u_map = {u: i for i, u in enumerate(pivot_temp.index)}
m_map = {m: i for i, m in enumerate(pivot_temp.columns)}

rng = np.random.default_rng(FIXED_SEED)
test_sample = test[['userId', 'movieId', 'rating', 'mean', 'std']].sample(n=min(3000, len(test)), random_state=FIXED_SEED)


# 3. 가중치 조합 탐색
for ALPHA in ALPHA_LIST:
    for BETA in BETA_LIST:
        GAMMA = 1 - ALPHA - BETA # 태그 가중치 (1 - α - β)
        
        # 가중치 합이 1을 초과하거나 음수 가중치가 발생하는 경우 제외
        if GAMMA < 0 or GAMMA > 1:
            continue

        print(f"\n--- Testing: R_w={ALPHA:.2f}, G_w={BETA:.2f}, T_w={GAMMA:.2f} ---")
        
        s = time.perf_counter()
        
        # [핵심] 3중 하이브리드 예측 함수 호출
        pred_matrix_z, u_ids, m_ids = fast_predict_matrix_triple_hybrid(
            train, S_genre_df, movie_tags_series, alpha=ALPHA, beta=BETA
        )
        
        e = time.perf_counter() - s
        
        # 4. 평가 (RMSE 계산)
        z_preds = []
        y_true = []

        for row in test_sample.itertuples():
            if row.userId in u_map and row.movieId in m_map:
                u_idx = u_map[row.userId]
                m_idx = m_map[row.movieId]
                pred_z = pred_matrix_z[u_idx, m_idx]
                
                # 역변환 및 클리핑
                raw_score = pred_z * row.std + row.mean
                final_score = np.clip(raw_score, 0.5, 5.0) 
                
                z_preds.append(final_score)
                y_true.append(row.rating)
        
        curr_rmse = rmse(y_true, z_preds)
        
        print(f"   -> RMSE: {curr_rmse:.4f} | Time: {e:.2f}s")
        
        grid_results.append({
            'alpha': ALPHA,
            'beta': BETA,
            'gamma': GAMMA,
            'rmse': curr_rmse
        })

# 5. 최적의 결과 출력
df_grid = pd.DataFrame(grid_results)
best_row = df_grid.loc[df_grid['rmse'].idxmin()]

print("\n=== Grid Search 결과 (RMSE 기준 상위 5개) ===")
print(df_grid.sort_values(by='rmse').head())

print("\n--- Best Combination ---")
print(f"Alpha (평점): {best_row['alpha']}")
print(f"Beta (장르): {best_row['beta']}")
print(f"Gamma (태그): {best_row['gamma']:.2f}")
print(f"Best RMSE: {best_row['rmse']:.4f}")

=== Grid Search 시작 (고정 시드 42 사용) ===

--- Testing: R_w=0.70, G_w=0.05, T_w=0.25 ---
   -> RMSE: 0.8943 | Time: 15.91s

--- Testing: R_w=0.70, G_w=0.10, T_w=0.20 ---
   -> RMSE: 0.9057 | Time: 11.77s

--- Testing: R_w=0.70, G_w=0.15, T_w=0.15 ---
   -> RMSE: 0.9115 | Time: 14.40s

--- Testing: R_w=0.80, G_w=0.05, T_w=0.15 ---
   -> RMSE: 0.8925 | Time: 13.98s

--- Testing: R_w=0.80, G_w=0.10, T_w=0.10 ---
   -> RMSE: 0.9045 | Time: 12.75s

--- Testing: R_w=0.80, G_w=0.15, T_w=0.05 ---
   -> RMSE: 0.9107 | Time: 17.42s

--- Testing: R_w=0.90, G_w=0.05, T_w=0.05 ---
   -> RMSE: 0.8913 | Time: 15.52s

=== Grid Search 결과 (RMSE 기준 상위 5개) ===
   alpha  beta  gamma      rmse
6    0.9  0.05   0.05  0.891335
3    0.8  0.05   0.15  0.892505
0    0.7  0.05   0.25  0.894296
4    0.8  0.10   0.10  0.904493
1    0.7  0.10   0.20  0.905660

--- Best Combination ---
Alpha (평점): 0.9
Beta (장르): 0.05
Gamma (태그): 0.05
Best RMSE: 0.8913


## 3. Multi-seed 실험 루프

In [8]:
# ==========================================
# [3. Multi-seed 실험 루프 - 3중 하이브리드 버전]
# ==========================================

results = []
best_reco = None

# 1. 초기화: 장르 유사도 행렬을 미리 계산
S_genre_df = calculate_genre_similarity(movies.copy()) 
movie_tags_series # 태그 시리즈는 1단계에서 이미 로드됨

# 2. 가중치 설정 (CF가 가장 중요하고 태그와 장르를 1:1로 보조)
ALPHA = 0.8  # 평점 가중치 (R_w)
BETA = 0.1   # 장르 가중치 (G_w)
GAMMA = 1 - ALPHA - BETA # 태그 가중치 (T_w) -> 0.1

print(f"하이브리드 실험 시작: R_w={ALPHA:.2f}, G_w={BETA:.2f}, T_w={GAMMA:.2f}. 총 {len(SEED_LIST)}개 시드 테스트.")

for idx, SEED in enumerate(SEED_LIST):
    print(f"\nRunning Seed {SEED} ({idx+1}/{len(SEED_LIST)})")

    # 1. 데이터 분할 및 전처리
    train_raw, test_raw = train_test_split(ratings, test_size=0.2, random_state=SEED)
    train, test, user_stats = run_preprocessing(train_raw, test_raw)

    # 2. [핵심] 3중 하이브리드 예측 함수 호출
    s = time.perf_counter()
    pred_matrix_z, u_ids, m_ids = fast_predict_matrix_triple_hybrid(
        train, S_genre_df, movie_tags_series, alpha=ALPHA, beta=BETA
    )
    e = time.perf_counter() - s
    
    # 인덱싱 맵핑
    u_map = {u: i for i, u in enumerate(u_ids)}
    m_map = {m: i for i, m in enumerate(m_ids)}

    # 3. Test 샘플링 (평가용 데이터 3000개)
    rng = np.random.default_rng(SEED)
    test_sample = test[['userId', 'movieId', 'rating', 'mean', 'std']].sample(n=min(3000, len(test)), random_state=SEED)

    # 4. Test 데이터에 대한 예측값 추출
    z_preds = []
    for uid, mid in zip(test_sample['userId'], test_sample['movieId']):
        if uid in u_map and mid in m_map:
            u_idx = u_map[uid]
            m_idx = m_map[mid]
            z_preds.append(pred_matrix_z[u_idx, m_idx])
        else:
            z_preds.append(0.0)

    # 5. 역변환 (클리핑 적용)
    original_preds = []
    for pred_z, row in zip(z_preds, test_sample.itertuples()):
        mean_u = row.mean
        std_u  = row.std
        raw_score = pred_z * std_u + mean_u
        final_score = np.clip(raw_score, 0.5, 5.0) 
        original_preds.append(final_score)

    # 6. 결과 저장
    avg_ms = (e / len(test_sample)) * 1000 
    
    res = {
        'seed': SEED,
        'rmse': rmse(test_sample['rating'], original_preds),
        'mae' : mae(test_sample['rating'], original_preds),
        'avg_ms': avg_ms
    }
    results.append(res)
    print(f" -> RMSE: {res['rmse']:.4f}, Time: {e:.4f}s")

    # 7. 첫 번째 시드(Seed 10)에서만 추천 결과 생성 
    if idx == 0:
        def recommend(user_id, topk=3):
            if user_id not in u_map: return None
            u_idx = u_map[user_id]
            user_row_preds = pred_matrix_z[u_idx] 
            watched = set(train[train['userId']==user_id]['movieId'])
            
            candidates = []
            for m_idx, m_id in enumerate(m_ids):
                if m_id in watched: continue
                
                z = user_row_preds[m_idx]
                mean_u = user_stats.loc[user_id, 'mean']
                std_u  = user_stats.loc[user_id, 'std']
                
                raw_score = z * std_u + mean_u
                final_score = np.clip(raw_score, 0.5, 5.0) 
                
                candidates.append((m_id, final_score))
            
            candidates.sort(key=lambda x: -x[1])
            top = candidates[:topk]
            
            return [{
                'movieId': m,
                'title': movies.loc[movies['movieId']==m, 'title'].iloc[0] if len(movies.loc[movies['movieId']==m]) > 0 else "Unknown",
                'predicted_rating': round(score, 3)
            } for m, score in top]

        print("\n[추천 결과 생성 중...]")
        best_reco = {
            'heavy': recommend(PERSONA_HEAVY_USER),
            'specialist': recommend(PERSONA_GENRE_SPECIALIST)
        }

print("\n실험 완료! (결과 요약 셀을 실행하세요)")

하이브리드 실험 시작: R_w=0.80, G_w=0.10, T_w=0.10. 총 5개 시드 테스트.

Running Seed 10 (1/5)
 -> RMSE: 0.8825, Time: 16.6319s

[추천 결과 생성 중...]

Running Seed 21 (2/5)
 -> RMSE: 0.9014, Time: 11.0486s

Running Seed 35 (3/5)
 -> RMSE: 0.8981, Time: 11.6287s

Running Seed 42 (4/5)
 -> RMSE: 0.9097, Time: 12.2880s

Running Seed 57 (5/5)
 -> RMSE: 0.8719, Time: 14.2329s

실험 완료! (결과 요약 셀을 실행하세요)


## 4. 결과 요약

In [9]:
import pandas as pd
df = pd.DataFrame(results)
print(df[['seed','rmse','mae','avg_ms']].round(4))
print("\n=== 평균 ± 표준편차 ===")
print(f"RMSE : {df.rmse.mean():.4f} ± {df.rmse.std():.4f}")
print(f"MAE  : {df.mae.mean():.4f} ± {df.mae.std():.4f}")
print(f"예측 속도 : {df.avg_ms.mean():.2f} ms (± {df.avg_ms.std():.2f})")

print("\n=== 페르소나 추천 결과 (Seed 10) ===")
print(f"헤비 유저 (userId={PERSONA_HEAVY_USER})")
for x in best_reco['heavy']:
    print(f"  → {x['title']}  (예측 평점 {x['predicted_rating']})")

print(f"\n드라마 전문가 (userId={PERSONA_GENRE_SPECIALIST})")
for x in best_reco['specialist']:
    print(f"  → {x['title']}  (예측 평점 {x['predicted_rating']})")

   seed    rmse     mae  avg_ms
0    10  0.8825  0.6813  5.5440
1    21  0.9014  0.6916  3.6829
2    35  0.8981  0.6986  3.8762
3    42  0.9097  0.7029  4.0960
4    57  0.8719  0.6730  4.7443

=== 평균 ± 표준편차 ===
RMSE : 0.8927 ± 0.0153
MAE  : 0.6895 ± 0.0123
예측 속도 : 4.39 ms (± 0.76)

=== 페르소나 추천 결과 (Seed 10) ===
헤비 유저 (userId=414)
  → La cravate (1957)  (예측 평점 4.679)
  → A Cosmic Christmas (1977)  (예측 평점 4.255)
  → The Adventures of Sherlock Holmes and Doctor Watson  (예측 평점 4.241)

드라마 전문가 (userId=85)
  → Far From Home: The Adventures of Yellow Dog (1995)  (예측 평점 5.0)
  → Swan Princess, The (1994)  (예측 평점 5.0)
  → Lassie (1994)  (예측 평점 5.0)


In [10]:
# ==========================================
# [Grid Search] 최적의 파라미터(태그 가중치, 개수) 찾기
# ==========================================
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import time

# 1. 테스트할 파라미터 후보군 정의
MAX_FEATURES_LIST = [500, 1000, 3000]   # 태그 개수 후보
TAG_WEIGHT_LIST   = [0.05, 0.1, 0.15, 0.2, 0.25] # 가중치 후보

# 결과 저장용
grid_results = []

print("=== Grid Search 시작 ===")

# Train/Test 데이터는 고정된 시드(예: 42)로 한 번만 나눔 (비교를 위해)
FIXED_SEED = 42
train_raw, test_raw = train_test_split(ratings, test_size=0.2, random_state=FIXED_SEED)
train, test, user_stats = run_preprocessing(train_raw, test_raw)

# [A] max_features 루프
for n_feat in MAX_FEATURES_LIST:
    print(f"\n[Testing] max_features = {n_feat} ...")
    
    # 1. TF-IDF 및 태그 유사도 계산 (n_feat가 바뀔 때마다 새로 계산)
    # 현재 Train에 있는 영화 ID 기준 태그 정렬
    pivot_temp = train.pivot(index='userId', columns='movieId', values='z_rating').fillna(0)
    movie_ids_temp = pivot_temp.columns
    current_movie_tags = [movie_tags_series.get(mid, "") for mid in movie_ids_temp]
    
    tfidf = TfidfVectorizer(stop_words='english', max_features=n_feat)
    tfidf_matrix = tfidf.fit_transform(current_movie_tags)
    sim_tags_fixed = linear_kernel(tfidf_matrix, tfidf_matrix) # 미리 계산
    
    # [B] tag_weight 루프
    for weight in TAG_WEIGHT_LIST:
        
        # --- 예측 로직 (Inline 구현) ---
        s_time = time.perf_counter()
        
        # 평점 유사도
        R = pivot_temp.values
        sim_cf = np.dot(R.T, R)
        max_cf = np.max(np.abs(sim_cf)) if np.max(np.abs(sim_cf)) > 0 else 1.0
        sim_cf_norm = sim_cf / max_cf
        
        # 하이브리드 결합
        sim_hybrid = (1 - weight) * sim_cf_norm + (weight) * sim_tags_fixed
        np.fill_diagonal(sim_hybrid, -np.inf)
        
        # 예측 계산
        S = np.where(sim_hybrid > 0, sim_hybrid, 0)
        num = np.dot(R, S)
        den = np.dot((R != 0).astype(float), np.abs(S))
        
        with np.errstate(divide='ignore', invalid='ignore'):
            pred_z = np.nan_to_num(num / den)
            
        # 평가 (Test Sample 3000개)
        rng = np.random.default_rng(FIXED_SEED)
        test_sample = test.sample(n=min(3000, len(test)), random_state=FIXED_SEED)
        
        u_map = {u: i for i, u in enumerate(pivot_temp.index)}
        m_map = {m: i for i, m in enumerate(pivot_temp.columns)}
        
        original_preds = []
        y_true = []
        
        for row in test_sample.itertuples():
            if row.userId in u_map and row.movieId in m_map:
                u_idx = u_map[row.userId]
                m_idx = m_map[row.movieId]
                val = pred_z[u_idx, m_idx]
                original_preds.append(val * row.std + row.mean)
                y_true.append(row.rating)
        
        curr_rmse = rmse(y_true, original_preds)
        e_time = time.perf_counter() - s_time
        
        print(f"   -> Weight: {weight} | RMSE: {curr_rmse:.4f} | Time: {e_time:.2f}s")
        
        grid_results.append({
            'max_features': n_feat,
            'tag_weight': weight,
            'rmse': curr_rmse
        })

# 3. 최적의 결과 출력
df_grid = pd.DataFrame(grid_results)
best_row = df_grid.loc[df_grid['rmse'].idxmin()]

print("\n=== 최적의 파라미터 조합 ===")
print(f"Max Features: {int(best_row['max_features'])}")
print(f"Tag Weight  : {best_row['tag_weight']}")
print(f"Best RMSE   : {best_row['rmse']:.4f}")

=== Grid Search 시작 ===

[Testing] max_features = 500 ...
   -> Weight: 0.05 | RMSE: 0.8843 | Time: 5.19s
   -> Weight: 0.1 | RMSE: 0.8853 | Time: 8.07s
   -> Weight: 0.15 | RMSE: 0.8863 | Time: 7.43s
   -> Weight: 0.2 | RMSE: 0.8875 | Time: 6.55s
   -> Weight: 0.25 | RMSE: 0.8888 | Time: 8.14s

[Testing] max_features = 1000 ...
   -> Weight: 0.05 | RMSE: 0.8844 | Time: 10.97s
   -> Weight: 0.1 | RMSE: 0.8853 | Time: 12.14s
   -> Weight: 0.15 | RMSE: 0.8863 | Time: 9.22s
   -> Weight: 0.2 | RMSE: 0.8874 | Time: 11.53s
   -> Weight: 0.25 | RMSE: 0.8886 | Time: 9.36s

[Testing] max_features = 3000 ...
   -> Weight: 0.05 | RMSE: 0.8845 | Time: 9.70s


KeyboardInterrupt: 